In [7]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer 
import re
from datetime import date, datetime, timedelta
from nltk.probability import FreqDist

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
df=pd.read_csv('news_clean_tokens.csv')
df.head(1)

,title,media,date,desc,link,Local,newtrititle,newtridesc
0,"['release', 'video', 'fatal', 'eve']",The Seattle Times,2019-01-01 00:00:00,"['faletogo', 'died', 'gunshot', 'wound', 'head...",https://www.seattletimes.com/seattle-news/crim...,0,"[('release', 'video', 'fatal'), ('video', 'fat...","[('faletogo', 'died', 'gunshot'), ('died', 'gu..."


## Topic Modeling 

In [11]:
df_topic =df[['title', 'desc','date']]
df_topic.head()

,title,desc,date
0,"['release', 'video', 'fatal', 'eve']","['faletogo', 'died', 'gunshot', 'wound', 'head...",2019-01-01 00:00:00
1,"['release', 'body', 'cam', 'footage', 'fatal',...","['released', 'body', 'cam', 'footage', 'involv...",2019-01-01 00:00:00
2,"['former', 'chief', 'jim', 'pugel', 'run', 'co...","['dozen', 'candidate', 'running', 'across', 's...",2019-01-01 00:00:00
3,"['judge', 'dismisses', 'claim', '2', 'fatal', ...","['lyles', 'history', 'report', 'domestic', 'di...",2019-01-01 00:00:00
4,"['dna', 'create', 'video', 'get', 'killer', 'c...","['dna', 'best', 'interest', 'u', 'immediately'...",2019-01-01 00:00:00


## KMeans clustering using scikit-learn

In [12]:
from sklearn import cluster, metrics

In [13]:
#extend list of stopwords to include search terms
stopwords = nltk.corpus.stopwords.words('english')
newStopWords = ['seattle','police','officer','officers', 'shooting', 'killed', 'spd', 'department', 'said', 'say','call',
               'city','according', 'man', 'woman', 'girl','boy','year', 'old', 'new', 'one', 'two','tip', 'line', '206', 'asking',
                'contact', '5000', '233']
#'dependable','professional','respectful','delivering','quality','enforces','prevents']
stopwords.extend(newStopWords)

In [14]:
df_date_dict = {'Q1': ['2019-01-01 00:00:00', '2019-02-01 00:00:00','2019-03-01 00:00:00'],
                'Q2':[ '2019-04-01 00:00:00', '2019-05-01 00:00:00', '2019-06-01 00:00:00'],
                'Q3':['2019-07-01 00:00:00', '2019-08-01 00:00:00', '2019-09-01 00:00:00'],
                'Q4':['2019-10-01 00:00:00','2019-11-01 00:00:00', '2019-12-01 00:00:00'],
                 'Q5':['2020-01-01 00:00:00', '2020-02-01 00:00:00','2020-03-01 00:00:00'],
                 'Q6':['2020-04-01 00:00:00', '2020-05-01 00:00:00', '2020-06-01 00:00:00'],
                 'Q7':['2020-07-01 00:00:00', '2020-08-01 00:00:00','2020-09-01 00:00:00'],
                 'Q8':['2020-10-01 00:00:00', '2020-11-01 00:00:00', '2020-12-01 00:00:00'],
                 'Q9':['2021-01-01 00:00:00', '2021-02-01 00:00:00', '2021-03-01 00:00:00'],
                 'Q10':['2021-04-01 00:00:00','2021-05-01 00:00:00', '2021-06-01 00:00:00'],
                 'Q11': ['2021-07-01 00:00:00']}

In [15]:
dataset =df_cluster=pd.read_csv('topic_mod_data.csv')

In [16]:
#for date_key in df_date_dict

def get_key_date(date_):
    for key_ in df_date_dict.keys():
        if date_ in df_date_dict[key_]:
            return key_
    return "key doesn't exist"
    
dataset['quarter']=[get_key_date(date_) for date_ in dataset['date']]

In [17]:
dataset.head(-5)

,title,media,date,desc,link,Local,quarter
0,Seattle police release video of fatal shooting...,The Seattle Times,2019-01-01 00:00:00,"Faletogo died of a gunshot wound to the head, ...",https://www.seattletimes.com/seattle-news/crim...,0,Q1
1,Seattle Police release body cam footage in fat...,MyNorthwest.com,2019-01-01 00:00:00,The Seattle Police Department released the bod...,https://mynorthwest.com/1232749/seattle-police...,0,Q1
2,Former Seattle police chief Jim Pugel will run...,The Seattle Times,2019-01-01 00:00:00,More than two dozen candidates are running acr...,https://www.seattletimes.com/seattle-news/poli...,0,Q1
3,Judge dismisses claims against 2 Seattle polic...,The Seattle Times,2019-01-01 00:00:00,"Lyles had a history of contacts with SPD, most...",https://www.seattletimes.com/seattle-news/crim...,0,Q1
4,‘We have your DNA’: Seattle police create vide...,The Seattle Times,2019-01-01 00:00:00,We have your DNA. It is in your best interest ...,https://www.seattletimes.com/seattle-news/crim...,0,Q1
...,...,...,...,...,...,...,...
2881,"Police can't solve all the beach problems, nei...",West Seattle Blog...,2021-07-01 00:00:00,Guests at the ACC meeting included a lieutenan...,https://westseattleblog.com/2021/07/police-can...,0,Q11
2882,One thought on “PubliCola Questions: Brianna T...,PubliCola,2021-07-01 00:00:00,What gaps in Seattle's non-police public safet...,https://publicola.com/2021/07/14/publicola-que...,0,Q11
2883,Uhaul truck explodes in Edmonds,KIRO-TV,2021-07-01 00:00:00,— A Uhaul truck exploded in Edmonds on Tuesday...,https://www.kiro7.com/news/local/uhaul-truck-e...,0,Q11
2884,‘Don’t have a clue’: It turns out Washington s...,The Seattle Times,2021-07-01 00:00:00,Seattle homicides increased 68% — from 31 in 2...,https://www.seattletimes.com/seattle-news/crim...,0,Q11


In [18]:
#Step the number of clusters
cluster_num=8

In [19]:

from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics

from sklearn.cluster import MiniBatchKMeans

import logging
from optparse import OptionParser
import sys
from time import time



# Display progress logs on stdout
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')

# parse commandline arguments
op = OptionParser()
op.add_option("--lsa",
              dest="n_components", type="int",
              help="Preprocess documents with latent semantic analysis.")
op.add_option("--no-minibatch",
              action="store_false", dest="minibatch", default=True,
              help="Use ordinary k-means algorithm (in batch mode).")
op.add_option("--no-idf",
              action="store_false", dest="use_idf", default=True,
              help="Disable Inverse Document Frequency feature weighting.")
op.add_option("--use-hashing",
              action="store_true", default=False,
              help="Use a hashing feature vectorizer")
op.add_option("--n-features", type=int, default=10000,
              help="Maximum number of features (dimensions)"
                   " to extract from text.")
op.add_option("--verbose",
              action="store_true", dest="verbose", default=False,
              help="Print progress reports inside k-means algorithm.")

print(__doc__)
op.print_help()


def is_interactive():
    return not hasattr(sys.modules['__main__'], '__file__')


# work-around for Jupyter notebook and IPython console
argv = [] if is_interactive() else sys.argv[1:]
(opts, args) = op.parse_args(argv)
if len(args) > 0:
    op.error("this script takes no arguments.")
    sys.exit(1)


# #############################################################################
# Load some categories from the training set

# Uncomment the following to do the analysis on all the categories
categories = None


#Clone of the cleaned original dataset, prior to preprocessing the text

stopwords = text.ENGLISH_STOP_WORDS.union(newStopWords)

print("%d documents" % len(dataset['desc']))

print()

labels = dataset['desc']
true_k = np.unique(labels).shape[0]

print("Extracting features from the training dataset "
      "using a sparse vectorizer")
t0 = time()
if opts.use_hashing:
    if opts.use_idf:
        # Perform an IDF normalization on the output of HashingVectorizer
        hasher = HashingVectorizer(n_features=opts.n_features,
                                   stop_words=stopwords, alternate_sign=False,
                                   norm=None)
        vectorizer = make_pipeline(hasher, TfidfTransformer())
    else:
        vectorizer = HashingVectorizer(n_features=opts.n_features,
                                       stop_words=stopwords,
                                       alternate_sign=False, norm='l2')
else:
    vectorizer = TfidfVectorizer(max_df=0.5, max_features=opts.n_features,
                                 min_df=2, stop_words=stopwords,
                                 use_idf=opts.use_idf)
X = vectorizer.fit_transform(dataset.desc)

print("done in %fs" % (time() - t0))
print("n_samples: %d, n_features: %d" % X.shape)
print()

if opts.n_components:
    print("Performing dimensionality reduction using LSA")
    t0 = time()
    # Vectorizer results are normalized, which makes KMeans behave as
    # spherical k-means for better results. Since LSA/SVD results are
    # not normalized, we have to redo the normalization.
    svd = TruncatedSVD(opts.n_components)
    normalizer = Normalizer(copy=False)
    lsa = make_pipeline(svd, normalizer)

    X = lsa.fit_transform(X)

    print("done in %fs" % (time() - t0))

    explained_variance = svd.explained_variance_ratio_.sum()
    print("Explained variance of the SVD step: {}%".format(
        int(explained_variance * 100)))

    print()


# #############################################################################
# Do the actual clustering

if opts.minibatch:
    km = MiniBatchKMeans(n_clusters=cluster_num, init='k-means++',
                          verbose=opts.verbose, random_state=42)
else:
    km = KMeans(n_clusters=cluster_num, init='k-means++', max_iter=200, 
                verbose=opts.verbose, random_state=42)

print("Clustering sparse data with %s" % km)
t0 = time()
km.fit(X)
print("done in %0.3fs" % (time() - t0))
print()

print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, km.labels_))
print("Completeness: %0.3f" % metrics.completeness_score(labels, km.labels_))
print("V-measure: %0.3f" % metrics.v_measure_score(labels, km.labels_))
print("Adjusted Rand-Index: %.3f"
      % metrics.adjusted_rand_score(labels, km.labels_))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, km.labels_))

print()


if not opts.use_hashing:
    print("Top terms per cluster:")

    if opts.n_components:
        original_space_centroids = svd.inverse_transform(km.cluster_centers_)
        order_centroids = original_space_centroids.argsort()[:, ::-1]
    else:
        order_centroids = km.cluster_centers_.argsort()[:, ::-1]

    terms = vectorizer.get_feature_names()
    for i in range(cluster_num):
        print("Cluster %d:" % i, end='')
        for ind in order_centroids[i, :20]:
            print(' %s' % terms[ind], end='')
        print()

Automatically created module for IPython interactive environment
Usage: ipykernel_launcher.py [options]

Options:
  -h, --help            show this help message and exit
  --lsa=N_COMPONENTS    Preprocess documents with latent semantic analysis.
  --no-minibatch        Use ordinary k-means algorithm (in batch mode).
  --no-idf              Disable Inverse Document Frequency feature weighting.
  --use-hashing         Use a hashing feature vectorizer
  --n-features=N_FEATURES
                        Maximum number of features (dimensions) to extract
                        from text.
  --verbose             Print progress reports inside k-means algorithm.
2891 documents

Extracting features from the training dataset using a sparse vectorizer
done in 0.310208s
n_samples: 2891, n_features: 3396

Clustering sparse data with MiniBatchKMeans(random_state=42, verbose=False)
done in 0.401s

Homogeneity: 0.136
Completeness: 1.000
V-measure: 0.239
Adjusted Rand-Index: 0.000
Silhouette Coefficient

In [20]:
df_topic['quarter']=dataset['quarter'].copy()

y=km.fit_predict(X)
df_topic['cluster']=y
df_topic

<ipython-input-20-0c5af4920761>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_topic['quarter']=dataset['quarter'].copy()
<ipython-input-20-0c5af4920761>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_topic['cluster']=y


,title,desc,date,quarter,cluster
0,"['release', 'video', 'fatal', 'eve']","['faletogo', 'died', 'gunshot', 'wound', 'head...",2019-01-01 00:00:00,Q1,5
1,"['release', 'body', 'cam', 'footage', 'fatal',...","['released', 'body', 'cam', 'footage', 'involv...",2019-01-01 00:00:00,Q1,0
2,"['former', 'chief', 'jim', 'pugel', 'run', 'co...","['dozen', 'candidate', 'running', 'across', 's...",2019-01-01 00:00:00,Q1,3
3,"['judge', 'dismisses', 'claim', '2', 'fatal', ...","['lyles', 'history', 'report', 'domestic', 'di...",2019-01-01 00:00:00,Q1,3
4,"['dna', 'create', 'video', 'get', 'killer', 'c...","['dna', 'best', 'interest', 'u', 'immediately'...",2019-01-01 00:00:00,Q1,3
...,...,...,...,...,...
2886,"['former', 'player', 'unlikely', 'participate'...","['asked', 'team', 'report', 'allegation', 'chi...",2021-07-01 00:00:00,Q11,3
2887,"['1', 'dead', '1', 'injured', 'federal', 'way']","['men', 'appear', 'known', 'dispute', 'shot', ...",2021-07-01 00:00:00,Q11,6
2888,"['detective', 'search', 'salem', 'ore', 'homic...","['anyone', 'see', 'soto', 'local', 'salem', '5...",2021-07-01 00:00:00,Q11,3
2889,"['investigate', 'death', 'female', 'inmate', '...","['lynnwood', 'inmate', 'present', 'died', 'kir...",2021-07-01 00:00:00,Q11,5


In [24]:
df_topic.to_csv('clean_tokens_clusters.csv')
df_topic.cluster.value_counts()

3    2128
4     158
6     132
2     129
5     111
0     107
7      72
1      54
Name: cluster, dtype: int64

In [22]:
dataset['cluster']=df_topic['cluster']
dataset.to_csv('original_df_clusters.csv')
dataset

,title,media,date,desc,link,Local,quarter,cluster
0,Seattle police release video of fatal shooting...,The Seattle Times,2019-01-01 00:00:00,"Faletogo died of a gunshot wound to the head, ...",https://www.seattletimes.com/seattle-news/crim...,0,Q1,5
1,Seattle Police release body cam footage in fat...,MyNorthwest.com,2019-01-01 00:00:00,The Seattle Police Department released the bod...,https://mynorthwest.com/1232749/seattle-police...,0,Q1,0
2,Former Seattle police chief Jim Pugel will run...,The Seattle Times,2019-01-01 00:00:00,More than two dozen candidates are running acr...,https://www.seattletimes.com/seattle-news/poli...,0,Q1,3
3,Judge dismisses claims against 2 Seattle polic...,The Seattle Times,2019-01-01 00:00:00,"Lyles had a history of contacts with SPD, most...",https://www.seattletimes.com/seattle-news/crim...,0,Q1,3
4,‘We have your DNA’: Seattle police create vide...,The Seattle Times,2019-01-01 00:00:00,We have your DNA. It is in your best interest ...,https://www.seattletimes.com/seattle-news/crim...,0,Q1,3
...,...,...,...,...,...,...,...,...
2886,Former player unlikely to participate in Black...,The Seattle Times,2021-07-01 00:00:00,He said he asked the team to report the allega...,https://www.seattletimes.com/sports/hockey/for...,0,Q11,3
2887,"1 dead, 1 injured in Federal Way shooting",The Seattle Times,2021-07-01 00:00:00,The two men appear to have known each other an...,https://www.seattletimes.com/seattle-news/crim...,0,Q11,6
2888,"Detectives search for Salem, Ore. homicide sus...",KIRO-TV,2021-07-01 00:00:00,Anyone who sees Soto should contact their loca...,https://www.kiro7.com/news/local/detectives-se...,0,Q11,3
2889,Police investigate death of female inmate at L...,The Seattle Times,2021-07-01 00:00:00,Lynnwood police said no other inmates were pre...,https://www.seattletimes.com/seattle-news/poli...,0,Q11,5


In [23]:
from sklearn.metrics.pairwise import pairwise_distances_argmin
from sklearn.datasets import make_blobs
import time


n_clusters=cluster_num
# #############################################################################
# Compute clustering with Means

k_means = KMeans(init='k-means++', n_clusters=cluster_num, n_init=10)
t0 = time.time()
k_means.fit(X)
t_batch = time.time() - t0

# #############################################################################
# Compute clustering with MiniBatchKMeans

mbk = MiniBatchKMeans(init='k-means++', n_clusters=cluster_num, batch_size=50,
                      n_init=10, max_no_improvement=10, verbose=0)
t0 = time.time()
mbk.fit(X)
t_mini_batch = time.time() - t0

# #############################################################################
# Plot result

fig = plt.figure(figsize=(8, 3))
fig.subplots_adjust(left=0.02, right=0.98, bottom=0.05, top=0.9)
colors = ['#4EACC5', '#FF9C34', '#4E9A06']

# We want to have the same colors for the same cluster from the
# MiniBatchKMeans and the KMeans algorithm. Let's pair the cluster centers per
# closest one.
k_means_cluster_centers = k_means.cluster_centers_
order = pairwise_distances_argmin(k_means.cluster_centers_,
                                  mbk.cluster_centers_)
mbk_means_cluster_centers = mbk.cluster_centers_[order]

k_means_labels = pairwise_distances_argmin(X, k_means_cluster_centers)
mbk_means_labels = pairwise_distances_argmin(X, mbk_means_cluster_centers)

# KMeans
ax = fig.add_subplot(1, 3, 1)
for k, col in zip(range(n_clusters), colors):
    my_members = k_means_labels == k
    cluster_center = k_means_cluster_centers[k]
    ax.plot(X[my_members, 0], X[my_members, 1], 'w',
            markerfacecolor=col, marker='.')
    ax.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col,
            markeredgecolor='k', markersize=6)
ax.set_title('KMeans')
ax.set_xticks(())
ax.set_yticks(())
plt.text(-3.5, 1.8,  'train time: %.2fs\ninertia: %f' % (
    t_batch, k_means.inertia_))

# MiniBatchKMeans
ax = fig.add_subplot(1, 3, 2)
for k, col in zip(range(n_clusters), colors):
    my_members = mbk_means_labels == k
    cluster_center = mbk_means_cluster_centers[k]
    ax.plot(X[my_members, 0], X[my_members, 1], 'w',
            markerfacecolor=col, marker='.')
    ax.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col,
            markeredgecolor='k', markersize=6)
ax.set_title('MiniBatchKMeans')
ax.set_xticks(())
ax.set_yticks(())
plt.text(-3.5, 1.8, 'train time: %.2fs\ninertia: %f' %
         (t_mini_batch, mbk.inertia_))

# Initialise the different array to all False
different = (mbk_means_labels == 4)
ax = fig.add_subplot(1, 3, 3)

for k in range(n_clusters):
    different += ((k_means_labels == k) != (mbk_means_labels == k))

identic = np.logical_not(different)
ax.plot(X[identic, 0], X[identic, 1], 'w',
        markerfacecolor='#bbbbbb', marker='.')
ax.plot(X[different, 0], X[different, 1], 'w',
        markerfacecolor='m', marker='.')
ax.set_title('Difference')
ax.set_xticks(())
ax.set_yticks(())

plt.show()

NameError: name 'KMeans' is not defined